In [43]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [44]:
wfh = pd.read_csv('/content/wfh.csv') # rate of employees working from home
quit_rate = pd.read_csv('/content/quit_data.csv') # employee quit rates
performance = pd.read_csv('/content/performance.csv') # employee performance

In [45]:
# Data Prep
quit_rate['live_far'].unique()
quit_rate['treat'] = quit_rate['treat'].apply(
    lambda x:1 if x == ' Treatment' else (0 if x == 'Control' else None))
quit_rate['role'] = quit_rate['role'].apply(
    lambda x: 1 if x == 'Manager' else (0 if x == 'Non-manager' else None))
quit_rate['male'] = quit_rate['male'].apply(
    lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
quit_rate['live_far'] = quit_rate['live_far'].apply(
    lambda x: 1 if x == ' Commute >  90 mins' else (
        0 if x == 'Commute <= 90 mins' else None))

## Question 1
**1.1 Assess the validity of the experiment.**

Randomization, according to the **concept**, is based on the birthday date. To challenge the assumption of randomization, one would need to identify a factor that is independent of the birthday-based assignment but correlated with the outcome (e.g., quit rate, performance). However, such a factor is unlikely to exist.

Volunteer: Both treatment groups have an equal mean value of **0.5**, demonstrating that the assignment to treatment is well-balanced with respect to the "volunteer" variable.

Role: The mean value for "role" is **0.247561** in the control group and **0.242424** in the treatment group. The difference is small, suggesting good balance between the groups.

Male: The proportion of males is slightly higher in the control group **0.654878** compared to the treatment group **0.637626**. The difference is small but noticeable.

Live_far: The proportion of employees living far from work shows minor variation between the control group **0.471698** and the treatment group **0.499197**, indicating reasonable balance.

Overall, according to the **data**, the variables appear reasonably balanced between the treatment and control groups, indicating that randomization was successful.

In [46]:
wfh.head()

treat  volunteer  takeuprate  date
0      0          0    0.004016     0
1      1          0    0.454183     0
2      0          1    0.001887     0
3      1          1    0.269450     0
4      0          0    0.004032     2

In [48]:
# Balance Check
# Balance check only use pre-treatment variables.
def balance_check(data, group_var, var):
    balance_table = data.groupby(group_var)[var].agg(
        mean='mean',
        std='std',
        N='count'
    )

    balance_table['sem'] = balance_table['std'] / np.sqrt(balance_table['N'])
    balance_table['ate'] = balance_table['mean'] - balance_table['mean'][0]
    balance_table['lift'] = balance_table['ate'] / balance_table['mean'][0]
    balance_table['se_ate'] = np.sqrt(balance_table['sem']**2 + balance_table['sem'][0]**2)
    balance_table['lb'] = balance_table['ate'] - 1.96 * balance_table['se_ate']
    balance_table['ub'] = balance_table['ate'] + 1.96 * balance_table['se_ate']
    balance_table['control_mean'] = balance_table['mean'][0]

    return balance_table[['mean', 'ate', 'lb', 'ub', 'lift']]

print(f"Balance check for employees working from home:\n{balance_check(wfh, 'treat', 'volunteer')}\n")
print(f"Balance check for quit rate (role):\n{balance_check(quit_rate, 'treat', 'role')}\n")
print(f"Balance check for quit rate (male):\n{balance_check(quit_rate, 'treat', 'male')}\n")
print(f"Balance check for quit rate (live_far):\n{balance_check(quit_rate, 'treat', 'live_far')}\n")
print(f"Balance check for quit rate:")
print(f"Since 'perf_2022h1' is not a pre-treatment variable, running the balance check is unnecessary.")

Balance check for employees working from home:
       mean  ate        lb        ub  lift
treat                                     
0       0.5  0.0 -0.180433  0.180433   0.0
1       0.5  0.0 -0.180433  0.180433   0.0

Balance check for quit rate (role):
           mean       ate        lb        ub      lift
treat                                                  
0      0.247561  0.000000 -0.041803  0.041803  0.000000
1      0.242424 -0.005137 -0.047157  0.036883 -0.020749

Balance check for quit rate (male):
           mean       ate        lb        ub      lift
treat                                                  
0      0.654878  0.000000 -0.046046  0.046046  0.000000
1      0.637626 -0.017252 -0.063967  0.029463 -0.026344

Balance check for quit rate (live_far):
           mean       ate        lb        ub      lift
treat                                                  
0      0.471698  0.000000 -0.054911  0.054911  0.000000
1      0.499197  0.027499 -0.027742  0.082741  0.0

In [60]:
quit_rate

treat  role  male  live_far  quit  role_dummy  male_dummy  far_dummy
0         1     1     1       0.0     0           0           0          0
1         1     1     1       1.0     0           0           0          0
2         0     0     1       0.0     0           0           0          0
3         0     0     1       1.0     0           0           0          0
4         0     0     1       0.0     0           0           0          0
...     ...   ...   ...       ...   ...         ...         ...        ...
1607      0     0     0       1.0     0           0           0          0
1608      1     0     0       1.0     0           0           0          0
1609      1     0     0       NaN     0           0           0          0
1610      0     1     1       1.0     0           0           0          0
1611      1     1     1       0.0     0           0           0          0

[1612 rows x 8 columns]

In [65]:
# Handle the Non-binary Variables
quit_rate['role_dummy'] = (quit_rate['role'] == 1).astype(int) # quit_rate = 'Manager'
quit_rate['male_dummy'] = (quit_rate['male'] == 1).astype(int) # quit_rate = 'Male'
quit_rate['far_dummy'] = (quit_rate['live_far'] == 1).astype(int) # quit_rate = Commute >  90 mins

# validation for randomization
balance_table_sample = quit_rate.groupby('treat').agg(
    role_mean=('role_dummy', 'mean'),
    role_std=('role_dummy', 'std'),
    male_mean=('male_dummy', 'mean'),
    male_std=('male_dummy', 'std'),
    live_far_mean=('far_dummy', 'mean'),
    live_far_std=('far_dummy', 'std'),
    quit_cnt=('treat', 'count'),
)

balance_table_sample

role_mean  role_std  male_mean  male_std  live_far_mean  live_far_std  \
treat                                                                          
0       0.247561  0.431859   0.654878  0.475698       0.365854      0.481963   
1       0.242424  0.428820   0.637626  0.480990       0.392677      0.488654   

       quit_cnt  
treat            
0           820  
1           792

**1.2 Is there reason to think we've solved the problem of confounding?**

The experiment is randomized, which ensures that the treatment assignment is independent of both observed and unobserved variables, thereby minimizing systematic differences between groups.
Additionally, the balance check results indicate that key variables such as volunteer, role, gender, and live_far are well-balanced across treatment groups, further supporting the effectiveness of randomization and thus reduce the probability of confounding.

**1.3 Did the intervention actually affect WFH rates?**

The ATE is **0.496166**, demonstrating a significant impact of the intervention on WFH rates.

For the **control group**, the confidence interval is **(-0.030290, 0.030290)**, which includes 0. This means the difference is not statistically significant.

For the **treatment group**, the confidence interval is **(0.465876, 0.526457)**, which does not include 0. This shows the difference is statistically significant.

The Standard Error of ATE is **0.015454**. This small standard error indicates a precise estimation of the treatment effect.

The TOT is **0.992941**, the Confidence interval of TOT is **(0.907248, 1.078635)**, indicating that the Treatment-on-the-Treated is also statistically significant.

To conclude, the intervention significantly affect WFH rates.

In [51]:
# Calculating the ATE and CI for WFH.
wfh_table_ate = wfh.groupby('treat')['takeuprate'].agg(
    mean = 'mean',
    sem = 'sem'
)

# Calculate the ATE and CI for quit rate
wfh_table_ate['ate'] = wfh_table_ate['mean'] - wfh_table_ate['mean'].iloc[0]
wfh_table_ate['se_ate'] = np.sqrt(wfh_table_ate['sem'].iloc[0]**2 + wfh_table_ate['sem'].iloc[1]**2)
wfh_table_ate['lb'] = wfh_table_ate['ate'] - 1.96*wfh_table_ate['se_ate']
wfh_table_ate['ub'] = wfh_table_ate['ate'] + 1.96*wfh_table_ate['se_ate']
cols_to_show_ate = ['mean', 'ate', 'lb', 'ub']
print(wfh_table_ate[cols_to_show_ate])

           mean       ate        lb        ub
treat                                        
0      0.003527  0.000000 -0.030290  0.030290
1      0.499694  0.496166  0.465876  0.526457


In [52]:
wfh_table = wfh.groupby('treat')['takeuprate'].agg(
    mean = 'mean',
    std = 'std',
    sem = 'sem',
    count = 'count'
)

# Calculate the ITT
wfh_table['itt'] = wfh_table['mean'] - wfh_table['mean'][0]
wfh_table['se_mean'] = wfh_table['std']/np.sqrt(wfh_table['count'])
wfh_table['se_itt'] = np.sqrt(wfh_table['se_mean']**2 + wfh_table['se_mean']**2)
wfh_table['lb'] = wfh_table['itt'] - 1.96*wfh_table['se_itt']
wfh_table['ub'] = wfh_table['itt'] + 1.96*wfh_table['se_itt']

cols_to_show = ['mean', 'se_mean', 'se_itt', 'lb', 'ub']
print(wfh_table[cols_to_show])

           mean   se_mean    se_itt        lb        ub
treat                                                  
0      0.003527  0.000432  0.000611 -0.001198  0.001198
1      0.499694  0.015448  0.021847  0.453346  0.538987


In [53]:
# Calculate TOT and CI for Treatment on Treated.
takeup = wfh[wfh['treat'] == 1]['takeuprate'].mean()
print(takeup)
wfh_table['tot'] = wfh_table['itt'] / takeup
wfh_table['tot_se'] = wfh_table['se_itt'] / takeup
wfh_table['tot_lb'] = wfh_table['tot'] - 1.96*wfh_table['tot_se']
wfh_table['tot_ub'] = wfh_table['tot'] + 1.96*wfh_table['tot_se']
cols_to_show_wfhtot = ['tot', 'tot_lb', 'tot_ub']
print(wfh_table[cols_to_show_wfhtot])

0.49969362333333334
            tot    tot_lb    tot_ub
treat                              
0      0.000000 -0.002398  0.002398
1      0.992941  0.907248  1.078635


# Question 2
**2.1 What does the experiment show for the effect of HWFH on quit rate and performance? Extra credit (+5%) Also calculate confidence intervals.**

The ATE for the quit rate is **-2.397**, suggesting that HWFH **reduced** the quit rate by 2.397% compared to the control group. However, the confidence interval for the quit rate is **(-4.710260, -0.084024)**, which does not include 0, indicating that the results are **statistically significant**.

The ATE for the performance is **-0.029957**, suggesting that HWFH **reduced** the performance by 2.9957% compared to the control group. However, the confidence interval for the performance is **(-0.079433, 0.019520)**, which includes 0, indicating that the results are **not statistically significant**.

For the quit rate for treatment group, TOT is **-4.797224**. Among participants who actually adopted the HWFH policy, quit rates decreased by approximately **4.8** percentage points. Confidence Interval for TOT is **(-9.426296, -0.168152)**. The interval does not include 0, confirming the statistical significance of the treatment effect for compliers.

For the performance for treatment group, TOT is **-0.05995**. Among participants who actually adopted the HWFH policy, quit rates decreased by approximately **6.0** percentage points. Confidence Interval for TOT is **(-0.158964  0.039064)**. The interval include 0, indicating that the results are **not statistically significant** for the treatment effect for compliers.

In [54]:
# Effect of HWFH on quit rate and performance?
# Extra credit (+5%) Also calculate confidence intervals.
quit_table = quit_rate.groupby('treat')['quit'].agg(
    mean = 'mean',
    std = 'std',
    sem = 'sem',
    count = 'count'
)

# Calculate the ITT for quit rate.
quit_table['itt'] = quit_table['mean'] - quit_table['mean'].iloc[0]
quit_table['se_mean'] = quit_table['std']/np.sqrt(quit_table['count'])
quit_table['se_itt'] = np.sqrt(quit_table['se_mean']**2 + quit_table['se_mean'][0]**2)
quit_table['lb'] = quit_table['itt'] - 1.96*quit_table['se_itt']
quit_table['ub'] = quit_table['itt'] + 1.96*quit_table['se_itt']

# Calculate the TOT for quit rate.
quit_table['tot'] = quit_table['itt'] / takeup
quit_table['tot_se'] = quit_table['se_itt'] / takeup
quit_table['tot_lb'] = quit_table['tot'] - 1.96*quit_table['tot_se']
quit_table['tot_ub'] = quit_table['tot'] + 1.96*quit_table['tot_se']
cols_to_show_qrtot = ['itt', 'lb', 'ub', 'tot', 'tot_lb', 'tot_ub']
print(f'Quit Rate Table:\n{quit_table[cols_to_show_qrtot]}')

Quit Rate Table:
            itt        lb        ub       tot    tot_lb    tot_ub
treat                                                            
0      0.000000 -2.502843  2.502843  0.000000 -5.008755  5.008755
1     -2.397142 -4.710260 -0.084024 -4.797224 -9.426296 -0.168152


In [55]:
# Effect of HWFH on performance
# print(performance['perf_2022h1'].unique())
grade_mapping = {'A': 4, 'B+': 3.5, 'B': 3, 'C': 2, 'D': 1}

performance['perf_2022h1_numeric'] = performance['perf_2022h1'].map(grade_mapping)
perf_table = performance.groupby('treat')['perf_2022h1_numeric'].agg(
    mean = 'mean',
    sem = 'sem'
)

# Calculate the ATE and CI for quit rate
perf_table['ate'] = perf_table['mean'] - perf_table['mean'].iloc[0]
perf_table['se_ate'] = np.sqrt(perf_table['sem'].iloc[0]**2 + perf_table['sem'].iloc[1]**2)
perf_table['lb'] = perf_table['ate'] - 1.96*perf_table['se_ate']
perf_table['ub'] = perf_table['ate'] + 1.96*perf_table['se_ate']
cols_to_show = ['mean', 'ate', 'lb', 'ub']
print(f'Performance Table:\n{perf_table[cols_to_show]}')

Performance Table:
           mean       ate        lb        ub
treat                                        
0      3.258443  0.000000 -0.049477  0.049477
1      3.228487 -0.029957 -0.079433  0.019520


In [56]:
# Effect of HWFH on performance
# print(performance['perf_2022h1'].unique())
grade_mapping = {'A': 4, 'B+': 3.5, 'B': 3, 'C': 2, 'D': 1}
performance['perf_2022h1_numeric'] = performance['perf_2022h1'].map(grade_mapping)
perf_table = performance.groupby('treat')['perf_2022h1_numeric'].agg(
    mean = 'mean',
    std = 'std',
    sem = 'sem',
    count = 'count'
)

# Calculate the ITT for performance.
perf_table['itt'] = perf_table['mean'] - perf_table['mean'][0]
perf_table['se_mean'] = perf_table['std']/np.sqrt(perf_table['count'])
perf_table['se_itt'] = np.sqrt(perf_table['se_mean']**2 + perf_table['se_mean'][0]**2)
perf_table['lb'] = perf_table['itt'] - 1.96*perf_table['se_itt']
perf_table['ub'] = perf_table['itt'] + 1.96*perf_table['se_itt']

# Calculate the TOT for performance.
perf_table['tot'] = perf_table['itt'] / takeup
perf_table['tot_se'] = perf_table['se_itt'] / takeup
perf_table['tot_lb'] = perf_table['tot'] - 1.96*perf_table['tot_se']
perf_table['tot_ub'] = perf_table['tot'] + 1.96*perf_table['tot_se']
cols_to_show_perftot = ['itt', 'lb', 'ub', 'tot', 'tot_lb', 'tot_ub']
print(f'Performance Table:\n{perf_table[cols_to_show_perftot]}')

Performance Table:
            itt        lb        ub      tot    tot_lb    tot_ub
treat                                                           
0      0.000000 -0.047505  0.047505  0.00000 -0.095067  0.095067
1     -0.029957 -0.079433  0.019520 -0.05995 -0.158964  0.039064


# Question 3
**3.1 What does the excludability condition require here?**

The excludability condition is that the only reason that affects your quit rate and performance is HWFH.

**3.2 How might it go wrong?**

Employees in the control group might feel frustrated or dissatisfied about not being granted the option to work from home, which could impact their quit rate and performance.

Also, if the employees know that work from home policy is implemented based on birth date, this might influence their perception of fairness and create unintended effects unrelated to the treatment itself.



# Question 4
**4.1 Is there any reason to be concerned about differential attrition when looking at quit rates? What about performance?**

The data table shows that the performance dataset contains 1,355 records, while the quit_rate dataset has 1,612 records. This discrepancy suggests the possibility of missing values in the performance data. Therefore, attrition should be accounted for in the performance dataset.

One possible reason that the control group could experience higher attrition due to dissatisfaction with the lack of a work-from-home option. For instance, some employees might have joined the study expecting the opportunity to work from home but, upon being placed in the control group, may decide to leave the experiment halfway.

In [57]:
perf_score = performance.groupby('treat')['perf_2022h1_numeric'].count()
print(perf_score, '\n')

total_employees = quit_rate.groupby('treat').size()
print(total_employees, '\n')

attr_perf = 1 - (perf_score / total_employees)
print(attr_perf, '\n')

diff_attr = attr_perf[1] - attr_perf[0]
print(f'The differential attirtion for performance table is {diff_attr}.')

treat
0    681
1    674
Name: perf_2022h1_numeric, dtype: int64 

treat
0    820
1    792
dtype: int64 

treat
0    0.169512
1    0.148990
dtype: float64 

The differential attirtion for performance table is -0.020522296132052253.


# Question 5
**5.1 What does the experiment show about the value of HWFH? Would you recommend scaling up HWFH?**

HWFH significantly reduces quit rates, highlighting its potential to improve employee retention. However, there is insufficient evidence to conclude that HWFH negatively impacts performance. Therefore, I recommend not scaling up HWFH at this stage and instead conducting additional experiments to gather more data.

**5.2 Is there any additional information you'd like, or further experiments you would run? (You should make a recommendation regardless.)**

Future experiments should incorporate confidence intervals (CI) to better evaluate the impact of HWFH on outcome metrics, such as performance and quit rates, across **different job roles**. By targeting specific roles, the experiments can identify which positions are better suited for remote work. For instance, roles like front desk staff, where in-person presence is critical, may not be suitable for HWFH. On the other hand, managerial roles or software engineers, who can perform their tasks remotely without compromising performance, could greatly benefit from HWFH.

# Question 6
**6.1 Extra credit  (25%) Using the Lee bounds trimming approach, construct bounds on the ITT (for performance).**

**6.2 What do the bounds imply for practical significance?**

The Lee bounds of -0.7715 to -0.02996 indicate that the treatment effect is consistently negative across all plausible scenarios, meaning the treatment is likely to have a detrimental impact on the outcome variable (perf_2022h1_numeric) compared to the control group.

The lower bound, -0.02996, is very close to zero, while the upper bound, -0.77151, is relatively large. This has significant implications for practical significance. The lower bound suggests that quit rates could almost be decreased without significantly negatively impacting performance. However, the upper bound indicates that a one percentage point increase in quit rate could lead to relatively large decrease in performance.

In [58]:
# Calculate differential attrition
print(f"Differential Attrition: {diff_attr:.2%}")

# Find the top/bottom 7% of control
control_sorted = performance[performance['treat'] == 0].sort_values(by = 'perf_2022h1_numeric')
upper_cut = int(len(control_sorted) * (1 - diff_attr))
mean_c_low = control_sorted.iloc[:upper_cut]['perf_2022h1_numeric'].mean()
mean_c = control_sorted['perf_2022h1_numeric'].mean()
mean_t = performance[performance['treat'] == 1]['perf_2022h1_numeric'].mean()
itt_upper = mean_t - mean_c_low
print(mean_c_low, mean_c)

lower_cut = int(len(control_sorted) * diff_attr)
mean_c_high = control_sorted.iloc[lower_cut:]['perf_2022h1_numeric'].mean()
itt_lower = mean_t - mean_c_high

print('Lee bounds: ', itt_lower, '-', itt_upper)

Differential Attrition: -2.05%
3.2584434654919234 3.2584434654919234
Lee bounds:  -0.771513353115727 - -0.029956818607650426
